# Init for pyqcu.

In [1]:
import cupy as cp
import numpy as np
from pyqcu import define
from pyqcu import io
from pyqcu import qcu
from pyqcu import eigen, cg
from pyqcu.set import params, argv, set_ptrs
params[define._NODE_RANK_] = define.rank
params[define._NODE_SIZE_] = define.size
kappa = 1 / (2 * argv[define._MASS_] + 8)
print('My rank is ', define.rank)
gauge_filename = f"quda_wilson-bistabcg-gauge_-{params[define._LAT_X_]}-{params[define._LAT_Y_]}-{params  [define._LAT_Z_]}-{params[define._LAT_T_]}-{params[define._LAT_XYZT_]}-{params[define._GRID_X_]}-{params[define._GRID_Y_]}-{params[define._GRID_Z_]}-{params[define._GRID_T_]}-{params[define._PARITY_]}-{params[define._NODE_RANK_]}-{params[define._NODE_SIZE_]}-{params[define._DAGGER_]}-f.h5"
print("Parameters:", params)


    @@@@@@######QCU NOTES START######@@@@@@@
    0. Required: MPI(e.g. 4.1.2), CUDA(e.g. 12.4), CMAKE(e.g. 3.22.1), GCC(e.g. 11.4.0), HDF5-MPI(e.g. 1.10.7,'apt install libhdf5-mpi-dev && export HDF5_MPI="ON" && pip install --no-binary=h5py h5py').
    1. The libqcu.so was compiled when pyqcu setup in download_path/PyQCU/lib, please add this path to your LD_LIBRARY_PATH.
    2. The QCU(PyQCU) splite grid by x->y->z->t, lattice by x->y->z->t->p->d->c->c or x->y->z->t->c->s(->p) and x->y->z->t->c->s->c->s(->p).
    3. The QUDA(PyQUDA) splite grid by t->z->y->x, lattice by c->c->x->y->z->t->p->d or c->s->x->y->z->t(->p) and c->s->c->s->x->y->z->t(->p).
    4. The QCU input params in numpy array(dtype=np.int32), argv in  numpy array(dtype=np.float32 or float64) array, set_ptrs in numpy array(dtype=np.int64), other in cupy array(dtype=cp.complex64 or complex128).
    5. The smallest lattice size is (x=4,y=4,z=4,t=8) that QCU support.
    @@@@@@######QCU NOTES END######@@@@@@@
    
Parameter

In [2]:
wilson_cg_params = params.copy()
wilson_cg_params[define._SET_INDEX_] = 0
wilson_cg_params[define._SET_PLAN_] = define._SET_PLAN1_
qcu.applyInitQcu(set_ptrs, wilson_cg_params, argv)

gridDim.x               :4096
blockDim.x              :128
host_params[_LAT_X_]    :16
host_params[_LAT_Y_]    :32
host_params[_LAT_Z_]    :32
host_params[_LAT_T_]    :32
host_params[_LAT_XYZT_] :524288
host_params[_GRID_X_]   :1
host_params[_GRID_Y_]   :1
host_params[_GRID_Z_]   :1
host_params[_GRID_T_]   :1
host_params[_PARITY_]   :0
host_params[_NODE_RANK_]:0
host_params[_NODE_SIZE_]:1
host_params[_DAGGER_]   :0
host_params[_MAX_ITER_] :10000
host_params[_SET_INDEX_]:0
host_params[_SET_PLAN_] :1
host_argv[_MASS_]       :0.000000e+00
host_argv[_TOL_]        :1.000000e-09
lat_2dim[_XY_]          :512
lat_2dim[_XZ_]          :512
lat_2dim[_XT_]          :512
lat_2dim[_YZ_]          :1024
lat_2dim[_YT_]          :1024
lat_2dim[_ZT_]          :1024
lat_3dim[_YZT_]         :32768
lat_3dim[_XZT_]         :16384
lat_3dim[_XYT_]         :16384
lat_3dim[_XYZ_]         :16384
lat_4dim                :524288
grid_2dim[_XY_]         :1
grid_2dim[_XZ_]         :1
grid_2dim[_XT_]         :1
grid_2

In [3]:
wilson_dslash_eo_params = params.copy()
wilson_dslash_eo_params[define._SET_INDEX_] = 1
wilson_dslash_eo_params[define._SET_PLAN_] = define._SET_PLAN0_
wilson_dslash_eo_params[define._PARITY_] = define._EVEN_
wilson_dslash_eo_params[define._DAGGER_] = define._NO_USE_
qcu.applyInitQcu(set_ptrs, wilson_dslash_eo_params, argv)

gridDim.x               :4096
blockDim.x              :128
host_params[_LAT_X_]    :16
host_params[_LAT_Y_]    :32
host_params[_LAT_Z_]    :32
host_params[_LAT_T_]    :32
host_params[_LAT_XYZT_] :524288
host_params[_GRID_X_]   :1
host_params[_GRID_Y_]   :1
host_params[_GRID_Z_]   :1
host_params[_GRID_T_]   :1
host_params[_PARITY_]   :0
host_params[_NODE_RANK_]:0
host_params[_NODE_SIZE_]:1
host_params[_DAGGER_]   :0
host_params[_MAX_ITER_] :10000
host_params[_SET_INDEX_]:1
host_params[_SET_PLAN_] :0
host_argv[_MASS_]       :0.000000e+00
host_argv[_TOL_]        :1.000000e-09
lat_2dim[_XY_]          :512
lat_2dim[_XZ_]          :512
lat_2dim[_XT_]          :512
lat_2dim[_YZ_]          :1024
lat_2dim[_YT_]          :1024
lat_2dim[_ZT_]          :1024
lat_3dim[_YZT_]         :32768
lat_3dim[_XZT_]         :16384
lat_3dim[_XYT_]         :16384
lat_3dim[_XYZ_]         :16384
lat_4dim                :524288
grid_2dim[_XY_]         :1
grid_2dim[_XZ_]         :1
grid_2dim[_XT_]         :1
grid_2

In [4]:
wilson_dslash_eo_dag_params = params.copy()
wilson_dslash_eo_dag_params[define._SET_INDEX_] = 2
wilson_dslash_eo_dag_params[define._SET_PLAN_] = define._SET_PLAN0_
wilson_dslash_eo_dag_params[define._PARITY_] = define._EVEN_
wilson_dslash_eo_dag_params[define._DAGGER_] = define._USE_
qcu.applyInitQcu(set_ptrs, wilson_dslash_eo_dag_params, argv)

gridDim.x               :4096
blockDim.x              :128
host_params[_LAT_X_]    :16
host_params[_LAT_Y_]    :32
host_params[_LAT_Z_]    :32
host_params[_LAT_T_]    :32
host_params[_LAT_XYZT_] :524288
host_params[_GRID_X_]   :1
host_params[_GRID_Y_]   :1
host_params[_GRID_Z_]   :1
host_params[_GRID_T_]   :1
host_params[_PARITY_]   :0
host_params[_NODE_RANK_]:0
host_params[_NODE_SIZE_]:1
host_params[_DAGGER_]   :1
host_params[_MAX_ITER_] :10000
host_params[_SET_INDEX_]:2
host_params[_SET_PLAN_] :0
host_argv[_MASS_]       :0.000000e+00
host_argv[_TOL_]        :1.000000e-09
lat_2dim[_XY_]          :512
lat_2dim[_XZ_]          :512
lat_2dim[_XT_]          :512
lat_2dim[_YZ_]          :1024
lat_2dim[_YT_]          :1024
lat_2dim[_ZT_]          :1024
lat_3dim[_YZT_]         :32768
lat_3dim[_XZT_]         :16384
lat_3dim[_XYT_]         :16384
lat_3dim[_XYZ_]         :16384
lat_4dim                :524288
grid_2dim[_XY_]         :1
grid_2dim[_XZ_]         :1
grid_2dim[_XT_]         :1
grid_2

In [5]:
wilson_dslash_oe_params = params.copy()
wilson_dslash_oe_params[define._SET_INDEX_] = 3
wilson_dslash_oe_params[define._SET_PLAN_] = define._SET_PLAN0_
wilson_dslash_oe_params[define._PARITY_] = define._ODD_
wilson_dslash_oe_params[define._DAGGER_] = define._NO_USE_
qcu.applyInitQcu(set_ptrs, wilson_dslash_oe_params, argv)

gridDim.x               :4096
blockDim.x              :128
host_params[_LAT_X_]    :16
host_params[_LAT_Y_]    :32
host_params[_LAT_Z_]    :32
host_params[_LAT_T_]    :32
host_params[_LAT_XYZT_] :524288
host_params[_GRID_X_]   :1
host_params[_GRID_Y_]   :1
host_params[_GRID_Z_]   :1
host_params[_GRID_T_]   :1
host_params[_PARITY_]   :1
host_params[_NODE_RANK_]:0
host_params[_NODE_SIZE_]:1
host_params[_DAGGER_]   :0
host_params[_MAX_ITER_] :10000
host_params[_SET_INDEX_]:3
host_params[_SET_PLAN_] :0
host_argv[_MASS_]       :0.000000e+00
host_argv[_TOL_]        :1.000000e-09
lat_2dim[_XY_]          :512
lat_2dim[_XZ_]          :512
lat_2dim[_XT_]          :512
lat_2dim[_YZ_]          :1024
lat_2dim[_YT_]          :1024
lat_2dim[_ZT_]          :1024
lat_3dim[_YZT_]         :32768
lat_3dim[_XZT_]         :16384
lat_3dim[_XYT_]         :16384
lat_3dim[_XYZ_]         :16384
lat_4dim                :524288
grid_2dim[_XY_]         :1
grid_2dim[_XZ_]         :1
grid_2dim[_XT_]         :1
grid_2

In [6]:
wilson_dslash_oe_dag_params = params.copy()
wilson_dslash_oe_dag_params[define._SET_INDEX_] = 4
wilson_dslash_oe_dag_params[define._SET_PLAN_] = define._SET_PLAN0_
wilson_dslash_oe_dag_params[define._PARITY_] = define._ODD_
wilson_dslash_oe_dag_params[define._DAGGER_] = define._USE_
qcu.applyInitQcu(set_ptrs, wilson_dslash_oe_dag_params, argv)

gridDim.x               :4096
blockDim.x              :128
host_params[_LAT_X_]    :16
host_params[_LAT_Y_]    :32
host_params[_LAT_Z_]    :32
host_params[_LAT_T_]    :32
host_params[_LAT_XYZT_] :524288
host_params[_GRID_X_]   :1
host_params[_GRID_Y_]   :1
host_params[_GRID_Z_]   :1
host_params[_GRID_T_]   :1
host_params[_PARITY_]   :1
host_params[_NODE_RANK_]:0
host_params[_NODE_SIZE_]:1
host_params[_DAGGER_]   :1
host_params[_MAX_ITER_] :10000
host_params[_SET_INDEX_]:4
host_params[_SET_PLAN_] :0
host_argv[_MASS_]       :0.000000e+00
host_argv[_TOL_]        :1.000000e-09
lat_2dim[_XY_]          :512
lat_2dim[_XZ_]          :512
lat_2dim[_XT_]          :512
lat_2dim[_YZ_]          :1024
lat_2dim[_YT_]          :1024
lat_2dim[_ZT_]          :1024
lat_3dim[_YZT_]         :32768
lat_3dim[_XZT_]         :16384
lat_3dim[_XYT_]         :16384
lat_3dim[_XYZ_]         :16384
lat_4dim                :524288
grid_2dim[_XY_]         :1
grid_2dim[_XZ_]         :1
grid_2dim[_XT_]         :1
grid_2

In [7]:
print("Set pointers:", set_ptrs)
print("Set pointers data:", set_ptrs.data)

Set pointers: [94231767935920 94231777723072 94231777898224 94231778076976
 94231778215248              0              0              0
              0              0]
Set pointers data: <memory at 0x7fe21890c100>


# Read from hdf5 files.

In [8]:
print("Gauge filename:", gauge_filename)
gauge = io.hdf5_xxxtzyx2grid_xxxtzyx(params, gauge_filename)
fermion_in_filename = gauge_filename.replace("gauge", "fermion-in")
print("Fermion in filename:", fermion_in_filename)
fermion_in = io.hdf5_xxxtzyx2grid_xxxtzyx(
    params, fermion_in_filename)
fermion_out_filename = gauge_filename.replace("gauge", "fermion-out")
print("Fermion out filename:", fermion_out_filename)
quda_fermion_out = io.hdf5_xxxtzyx2grid_xxxtzyx(
    params, fermion_out_filename)
fermion_out = cp.zeros_like(fermion_in)
print("Fermion out data:", fermion_out.data)
print("Fermion out shape:", fermion_out.shape)
eigenvalues_filename = gauge_filename.replace("gauge", "eigenvalues")
print("Eigenvalues filename:", eigenvalues_filename)
eigenvalues = io.hdf5_xxx2xxx(file_name=eigenvalues_filename)
print("Eigenvalues data:", eigenvalues.data)
print("Eigenvalues shape:", eigenvalues.shape)
eigenvectors_filename = gauge_filename.replace("gauge", "eigenvectors")
print("Eigenvectors filename:", eigenvectors_filename)
eigenvectors = io.eigenvectors2esctzyx(
    params=params, eigenvectors=io.hdf5_xxx2xxx(file_name=eigenvectors_filename))
print("Eigenvectors data:", eigenvectors.data)
print("Eigenvectors shape:", eigenvectors.shape)

Gauge filename: quda_wilson-bistabcg-gauge_-32-32-32-32-1048576-1-1-1-1-0-0-1-0-f.h5
Grid Index T: 0, Grid Index Z: 0, Grid Index Y: 0, Grid Index X: 0
Grid Lat T: 32, Grid Lat Z: 32, Grid Lat Y: 32, Grid Lat X: 16
All Dest Shape: (3, 3, 4, 2, 32, 32, 32, 16)
Dest Shape: (3, 3, 4, 2, 32, 32, 32, 16)
Fermion in filename: quda_wilson-bistabcg-fermion-in_-32-32-32-32-1048576-1-1-1-1-0-0-1-0-f.h5
Grid Index T: 0, Grid Index Z: 0, Grid Index Y: 0, Grid Index X: 0
Grid Lat T: 32, Grid Lat Z: 32, Grid Lat Y: 32, Grid Lat X: 16
All Dest Shape: (2, 4, 3, 32, 32, 32, 16)
Dest Shape: (2, 4, 3, 32, 32, 32, 16)
Fermion out filename: quda_wilson-bistabcg-fermion-out_-32-32-32-32-1048576-1-1-1-1-0-0-1-0-f.h5
Grid Index T: 0, Grid Index Z: 0, Grid Index Y: 0, Grid Index X: 0
Grid Lat T: 32, Grid Lat Z: 32, Grid Lat Y: 32, Grid Lat X: 16
All Dest Shape: (2, 4, 3, 32, 32, 32, 16)
Dest Shape: (2, 4, 3, 32, 32, 32, 16)
Fermion out data: <MemoryPointer 0xb28400000 device=0 mem=<cupy.cuda.memory.PooledMemor

In [9]:
set_ptrs

array([94231767935920, 94231777723072, 94231777898224, 94231778076976,
       94231778215248,              0,              0,              0,
                    0,              0])

# Run wilson bistabcg from pyqcu test.

In [10]:
# qcu.applyWilsonBistabCgQcu(fermion_out, fermion_in,
#                            gauge, set_ptrs, wilson_cg_params)
qcu.applyWilsonCgQcu(fermion_out, fermion_in,
                           gauge, set_ptrs, wilson_cg_params)
print("Fermion out data:", fermion_out.data)
print("Fermion out shape:", fermion_out.shape)
print("QUDA Fermion out data:", quda_fermion_out.data)
print("QUDA Fermion out shape:", quda_fermion_out.shape)
print("Difference:", cp.linalg.norm(fermion_out -
      quda_fermion_out)/cp.linalg.norm(quda_fermion_out))

##RANK:0##LOOP:147##Residual:(6.68707e-10,-4.222e-21i)
multi-gpu wilson Cg total time: (without malloc free memcpy) :1.764982400 sec
######TIME  :5800.38######
##RANK      :0
##LOOP      :-2
##tmp0      :(6.79706e-10,1.0842e-19i)
##rho_prev  :(1.27738e-09,8.45858e-21i)
##rho       :(6.68707e-10,-4.222e-21i)
##alpha     :(1.87931,-2.87325e-10i)
##beta      :(0.523499,-6.77173e-12i)
##send_tmp  :(0.0073078,0i)
##norm2_tmp :(492090,-3.55977e-06i)
##diff_tmp  :(1.48505e-08,1.07428e-19i)
##lat_4dim  :(524288,0i)
Fermion out data: <MemoryPointer 0xb28400000 device=0 mem=<cupy.cuda.memory.PooledMemory object at 0x7fe218881530>>
Fermion out shape: (2, 4, 3, 32, 32, 32, 16)
QUDA Fermion out data: <MemoryPointer 0xb22400000 device=0 mem=<cupy.cuda.memory.PooledMemory object at 0x7fe2902cadb0>>
QUDA Fermion out shape: (2, 4, 3, 32, 32, 32, 16)
Difference: 3.4667485e-07


# Run matvec(eigenvector[.]) ?= eigenvalue[.]*eigenvector[.] for eigen test

In [11]:
# src_o-set_ptr->kappa()**2*dslash_oe(dslash_eo(src_o))
def cg_dslash_no_dag(src):
    tmp0 = cp.zeros_like(src)
    tmp1 = cp.zeros_like(src)
    qcu.applyWilsonDslashQcu(
        tmp0, src, gauge, set_ptrs, wilson_dslash_eo_params)
    qcu.applyWilsonDslashQcu(
        tmp1, tmp0, gauge, set_ptrs, wilson_dslash_oe_params)
    return src-kappa**2*tmp1


def cg_dslash_dag(src):
    tmp0 = cp.zeros_like(src)
    tmp1 = cp.zeros_like(src)
    qcu.applyWilsonDslashQcu(
        tmp0, src, gauge, set_ptrs, wilson_dslash_eo_dag_params)
    qcu.applyWilsonDslashQcu(
        tmp1, tmp0, gauge, set_ptrs, wilson_dslash_oe_dag_params)
    return src-kappa**2*tmp1


def cg_dslash(src):
    return cg_dslash_dag(cg_dslash_no_dag(src))

cg_dslash(fermion_in[0])

multi-gpu wilson dslash total time: (without malloc free memcpy) :0.001853420 sec
multi-gpu wilson dslash total time: (without malloc free memcpy) :0.001768219 sec
multi-gpu wilson dslash total time: (without malloc free memcpy) :0.001862707 sec
multi-gpu wilson dslash total time: (without malloc free memcpy) :0.001784401 sec


array([[[[[[ 9.51220319e-02-1.50650978e-01j,
             2.45946553e-02-1.02814257e-01j,
             1.48024231e-01-4.47862744e-02j, ...,
             7.41664469e-02+9.95192677e-04j,
             5.30980304e-02-3.66091430e-02j,
             3.46460044e-02-3.93299088e-02j],
           [-8.17881525e-03-3.08910906e-02j,
             3.48504521e-02+1.36266977e-01j,
            -1.22746021e-01-6.90166056e-02j, ...,
            -1.25927895e-01+1.79068416e-01j,
             1.41903013e-03+4.17234413e-02j,
            -8.24942142e-02-7.95281306e-02j],
           [ 6.73036128e-02-3.39578688e-02j,
            -1.60460129e-01+1.05337277e-02j,
            -4.40900363e-02+2.68261492e-01j, ...,
             6.13424480e-02-5.78982756e-03j,
             1.40210494e-01+4.27314751e-02j,
            -1.11338377e-01+1.34403389e-02j],
           ...,
           [ 7.02213421e-02-6.88914359e-02j,
             9.23898667e-02+1.44228742e-01j,
            -1.88732073e-02-1.99812483e-02j, ...,
             1.9

In [12]:
def matvec(src):
    dest = cp.zeros_like(src)
    qcu.applyWilsonCgDslashQcu(
        dest, src, gauge, set_ptrs, wilson_cg_params)
    return dest

matvec(fermion_in[0])

array([[[[[[ 9.51220319e-02-1.50650978e-01j,
             2.45946553e-02-1.02814257e-01j,
             1.48024231e-01-4.47862744e-02j, ...,
             7.41664469e-02+9.95192677e-04j,
             5.30980304e-02-3.66091430e-02j,
             3.46460044e-02-3.93299088e-02j],
           [-8.17881525e-03-3.08910906e-02j,
             3.48504521e-02+1.36266977e-01j,
            -1.22746021e-01-6.90166056e-02j, ...,
            -1.25927895e-01+1.79068416e-01j,
             1.41903013e-03+4.17234413e-02j,
            -8.24942142e-02-7.95281306e-02j],
           [ 6.73036128e-02-3.39578688e-02j,
            -1.60460129e-01+1.05337277e-02j,
            -4.40900363e-02+2.68261492e-01j, ...,
             6.13424480e-02-5.78982756e-03j,
             1.40210494e-01+4.27314751e-02j,
            -1.11338377e-01+1.34403389e-02j],
           ...,
           [ 7.02213421e-02-6.88914359e-02j,
             9.23898667e-02+1.44228742e-01j,
            -1.88732073e-02-1.99812483e-02j, ...,
             1.9

In [13]:
fermion_in[0]

array([[[[[[1.+1.j, 1.+1.j, 1.+1.j, ..., 1.+1.j, 1.+1.j, 1.+1.j],
           [1.+1.j, 1.+1.j, 1.+1.j, ..., 1.+1.j, 1.+1.j, 1.+1.j],
           [1.+1.j, 1.+1.j, 1.+1.j, ..., 1.+1.j, 1.+1.j, 1.+1.j],
           ...,
           [1.+1.j, 1.+1.j, 1.+1.j, ..., 1.+1.j, 1.+1.j, 1.+1.j],
           [1.+1.j, 1.+1.j, 1.+1.j, ..., 1.+1.j, 1.+1.j, 1.+1.j],
           [1.+1.j, 1.+1.j, 1.+1.j, ..., 1.+1.j, 1.+1.j, 1.+1.j]],

          [[1.+1.j, 1.+1.j, 1.+1.j, ..., 1.+1.j, 1.+1.j, 1.+1.j],
           [1.+1.j, 1.+1.j, 1.+1.j, ..., 1.+1.j, 1.+1.j, 1.+1.j],
           [1.+1.j, 1.+1.j, 1.+1.j, ..., 1.+1.j, 1.+1.j, 1.+1.j],
           ...,
           [1.+1.j, 1.+1.j, 1.+1.j, ..., 1.+1.j, 1.+1.j, 1.+1.j],
           [1.+1.j, 1.+1.j, 1.+1.j, ..., 1.+1.j, 1.+1.j, 1.+1.j],
           [1.+1.j, 1.+1.j, 1.+1.j, ..., 1.+1.j, 1.+1.j, 1.+1.j]],

          [[1.+1.j, 1.+1.j, 1.+1.j, ..., 1.+1.j, 1.+1.j, 1.+1.j],
           [1.+1.j, 1.+1.j, 1.+1.j, ..., 1.+1.j, 1.+1.j, 1.+1.j],
           [1.+1.j, 1.+1.j, 1.+1.j, ...,

In [14]:

for i, ev in enumerate(eigenvalues):
    print(f"λ_{i} = {ev:.2e}")
    # Verify eigenvector
    v = eigenvectors[i]
    w = cp.zeros_like(v)
    w = matvec(v)
    error = cp.linalg.norm(w - ev * v) / cp.linalg.norm(w)
    print(f"Relative error: {error:.2e}")
    j = i+1
    if j == len(eigenvalues):
        j = 0
    print(
        f"Diff between λ_{i} and λ_{j}: {cp.linalg.norm(eigenvectors[i] - eigenvectors[j])/cp.linalg.norm(eigenvectors[i]):.2e}")

λ_0 = 6.46e-04+0.00e+00j
Relative error: 8.08e-03
Diff between λ_0 and λ_1: 1.41e+00
λ_1 = 6.46e-04+0.00e+00j
Relative error: 9.35e-03
Diff between λ_1 and λ_2: 1.41e+00
λ_2 = 6.46e-04+0.00e+00j
Relative error: 1.03e-02
Diff between λ_2 and λ_3: 1.41e+00
λ_3 = 6.46e-04+0.00e+00j
Relative error: 9.62e-03
Diff between λ_3 and λ_4: 1.41e+00
λ_4 = 6.46e-04+0.00e+00j
Relative error: 1.00e-02
Diff between λ_4 and λ_5: 1.41e+00
λ_5 = 6.46e-04+0.00e+00j
Relative error: 9.52e-03
Diff between λ_5 and λ_6: 1.41e+00
λ_6 = 6.46e-04+0.00e+00j
Relative error: 9.83e-03
Diff between λ_6 and λ_7: 1.41e+00
λ_7 = 6.46e-04+0.00e+00j
Relative error: 9.38e-03
Diff between λ_7 and λ_8: 1.41e+00
λ_8 = 6.46e-04+0.00e+00j
Relative error: 9.36e-03
Diff between λ_8 and λ_9: 1.41e+00
λ_9 = 6.46e-04+0.00e+00j
Relative error: 1.06e-02
Diff between λ_9 and λ_10: 1.41e+00
λ_10 = 6.46e-04+0.00e+00j
Relative error: 1.00e-02
Diff between λ_10 and λ_11: 1.41e+00
λ_11 = 6.46e-04+0.00e+00j
Relative error: 1.12e-02
Diff betwe

# Give guage's eigenvalues and eigenvectors to hdf5 files. (pass, don't run this)

In [15]:
# eigen_solver = eigen.solver(
#     n=params[define._LAT_XYZT_] * define._LAT_HALF_SC_, k=define._LAT_E_,matvec=matvec,dtype=gauge.dtype)
# eigenvalues, eigenvectors = eigen_solver.run()
# io.xxx2hdf5_xxx(
#     eigenvalues, params, gauge_filename.replace("gauge", "eigenvalues"))
# io.xxx2hdf5_xxx(
#     eigenvectors, params, gauge_filename.replace("gauge", "eigenvectors"))

# MultiGrid - R*vector

In [16]:
fermion_in = io.xxxtzyx2mg_xxxtzyx(input_array=fermion_in, params=params)
eigenvectors = io.xxxtzyx2mg_xxxtzyx(input_array=eigenvectors, params=params)

Input Array Shape: (2, 4, 3, 32, 32, 32, 16)
Dest Shape: (2, 4, 3, 4, 8, 2, 16, 2, 16, 2, 8)
Input Array Shape: (24, 4, 3, 32, 32, 32, 16)
Dest Shape: (24, 4, 3, 4, 8, 2, 16, 2, 16, 2, 8)


# Origin CG

In [17]:
b_e = fermion_in[define._EVEN_].flatten()
b_o = fermion_in[define._ODD_].flatten()
b__o = cp.zeros_like(b_o)
tmp = cp.zeros_like(b_o)
# b__o=b_o+kappa*D_oe(b_e)
qcu.applyWilsonDslashQcu(tmp, b_e, gauge, set_ptrs, wilson_dslash_oe_params)
b__o = b_o+kappa*tmp
# b__o -> Dslash^dag b__o
b__o = cg_dslash_dag(b__o)


multi-gpu wilson dslash total time: (without malloc free memcpy) :0.001916120 sec
multi-gpu wilson dslash total time: (without malloc free memcpy) :0.001891220 sec
multi-gpu wilson dslash total time: (without malloc free memcpy) :0.001889303 sec


In [18]:
b__o

array([ 0.06233358+0.15173662j,  0.32942653+0.06118965j,
       -0.04789817-0.00079036j, ...,  0.11167705+0.35566437j,
        0.00184178+0.01615477j,  0.27256525-0.02667701j], dtype=complex64)

In [19]:
# Dslash(x_o)=b__o
cg_solver = cg.slover(b=b__o, matvec=matvec, tol=1e-10, max_iter=1000000)
x_o = cg_solver.run()
# x_e  =b_e+kappa*D_eo(x_o)
qcu.applyWilsonDslashQcu(tmp, x_o, gauge, set_ptrs, wilson_dslash_eo_params)
x_e = b_e+kappa*tmp
# give qcu_fermion_out
qcu_fermion_out = cp.zeros_like(quda_fermion_out)
qcu_fermion_out[define._EVEN_] = x_e.reshape(
    quda_fermion_out[define._EVEN_].shape)
qcu_fermion_out[define._ODD_] = x_o.reshape(
    quda_fermion_out[define._ODD_].shape)

Iteration 0: Residual = 4.707547e+05, Time = 0.018075 s
Iteration 1: Residual = 8.638923e+04, Time = 0.014930 s
Iteration 2: Residual = 8.538588e+04, Time = 0.014929 s
Iteration 3: Residual = 2.528403e+05, Time = 0.014271 s
Iteration 4: Residual = 7.458902e+05, Time = 0.013782 s
Iteration 5: Residual = 1.788648e+06, Time = 0.013788 s
Iteration 6: Residual = 3.071307e+06, Time = 0.013712 s
Iteration 7: Residual = 3.649306e+06, Time = 0.020833 s
Iteration 8: Residual = 3.262904e+06, Time = 0.014936 s
Iteration 9: Residual = 2.303283e+06, Time = 0.013723 s
Iteration 10: Residual = 1.464573e+06, Time = 0.017502 s
Iteration 11: Residual = 8.926068e+05, Time = 0.021734 s
Iteration 12: Residual = 5.376489e+05, Time = 0.019506 s
Iteration 13: Residual = 3.305778e+05, Time = 0.020795 s
Iteration 14: Residual = 2.062932e+05, Time = 0.016908 s
Iteration 15: Residual = 1.322379e+05, Time = 0.014195 s
Iteration 16: Residual = 8.682344e+04, Time = 0.013502 s
Iteration 17: Residual = 5.835934e+04, Ti

In [20]:
np.linalg.norm(qcu_fermion_out-quda_fermion_out) / \
    np.linalg.norm(quda_fermion_out)

array(3.471956e-07, dtype=float32)

# End for pyqcu. (pass, don't run this)

In [21]:
# qcu.applyEndQcu(set_ptrs, params)
# qcu.applyEndQcu(set_ptrs, wilson_dslash_eo_params)
# qcu.applyEndQcu(set_ptrs, wilson_dslash_oe_params)